![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [11]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = './Dados/UCI HAR Dataset/features.txt'
filename_labels = "./Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "./Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "./Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "./Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "./Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "./Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "./Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze()
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze()
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze()
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

# alterei alguns nomes e métodos para funcionar melhor.
# o parâmetro squeeze parece não existir mais dentro do método ```pd.read_csv```portanto, coloquei-o  no encadeamento para rodar.

In [89]:
X_train.shape # 561 variáveis no original

(7352, 561)

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [40]:
%%time
# criando árvore com os parâmetros solicitados de alpha = o.oo1 e fitando com a base de treino
tree = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(X_train, y_train)

# visualizando a acurácia
train_score = tree.score(X_train, y_train)
test_score = tree.score(X_test, y_test)

print(f'Acurácia da melhor árvore na base de treino:    {train_score*100:.1f}')
print(f'Acurácia da melhor árvore na base de teste:     {test_score *100:.1f}')

Acurácia da melhor árvore na base de treino:    97.6
Acurácia da melhor árvore na base de teste:     88.0
CPU times: total: 8.66 s
Wall time: 8.73 s


> O tempo de processamento foi curto se comparado a outras bases de dados, somente 8.66 s.

> A acurácia está alta para ambos treino e teste.

## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [38]:
%%time

# PCA com 1 componente
prcomp = PCA(n_components=1).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

print(pc_treino.shape)

(7352, 1)
CPU times: total: 500 ms
Wall time: 144 ms


In [36]:
# definindo a quantidade de componentes que necessitam de nomes de colunas
n=1

# criando o nome das colunas com um loop
colunas = ['cp'+str(x+1) for x in list(range(n))]

# aplicando o nome nas colunas e criando a DF com apenas o componente escolhido 
pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
pc_test  = pd.DataFrame( pc_teste[:,:n], columns = colunas)


pc_train.head()

,cp1
0,-5.520280
1,-5.535350
2,-5.474988
3,-5.677232
4,-5.748749


In [37]:
# criando nova árvore classificatória
# Usei o mesmo ccp_alpha para criar esta árvore classificada.
# Fit na árvore com o pc_train que vem com a DF que sofreu o PCA como variável explicativa

clf = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_train, y_train)

In [43]:
# Avaliando a nova acurácia

train_score_pca = clf.score(pc_train, y_train)
test_score_pca = clf.score(pc_test, y_test)

print(f'Acurácia da melhor árvore na base de treino:    {train_score*100:.1f}')
print(f'Acurácia da melhor árvore na base de treino com PCA:    {train_score_pca*100:.1f}')
print(f'Acurácia da melhor árvore na base de teste:     {test_score *100:.1f}')
print(f'Acurácia da melhor árvore na base de teste com PCA:     {test_score_pca *100:.1f}')

Acurácia da melhor árvore na base de treino:    97.6
Acurácia da melhor árvore na base de treino com PCA:    50.0
Acurácia da melhor árvore na base de teste:     88.0
Acurácia da melhor árvore na base de teste com PCA:     45.7


> O tempo de processamento caiu bastante, deixando o processo muito mais rápido.

> No entanto, com o componente único, a acurácia diminuiu em ambas bases.

## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [88]:
%%time

# grid search com a lista de componentes desejada

cmpts = [1, 2, 5, 10, 50]

for cmpt in cmpts:
    prcomp = PCA(n_components=cmpt).fit(X_train) 
    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)

    # criando o nome das colunas com um loop
    n = int(cmpt)
    colunas = ['cp'+str(x+1) for x in list(range(n))]

    # aplicando o nome nas colunas e criando a DF com apenas o componente escolhido 
    pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
    pc_test  = pd.DataFrame( pc_teste[:,:n], columns = colunas)
    
    #criando e fitando a árvore na base pc com a quantidade de componentes desejada
    # mantendo o mesmo alpha e mesmo randon_state
    clf = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_train, y_train)

    # obtendo a acurácia de cada uma das árvores com seus respectivos componentes
    train_score_pca = clf.score(pc_train, y_train)
    test_score_pca = clf.score(pc_test, y_test)

    print(f'Acurácia base de treino, componentes = {cmpt}:    {train_score_pca*100:.1f}')
    print(f'Acurácia base de teste, componentes = {cmpt}:     {test_score_pca *100:.1f}')
    print(f'--')
    
print(f'561 componentes')
print(f'Acurácia original na base de treino:    {train_score*100:.1f}')
print(f'Acurácia original na base de teste:     {test_score *100:.1f}')
    

Acurácia base de treino, componentes = 1:    50.0
Acurácia base de teste, componentes = 1:     45.7
--
Acurácia base de treino, componentes = 2:    61.3
Acurácia base de teste, componentes = 2:     58.5
--
Acurácia base de treino, componentes = 5:    84.6
Acurácia base de teste, componentes = 5:     78.9
--
Acurácia base de treino, componentes = 10:    89.3
Acurácia base de teste, componentes = 10:     82.3
--
Acurácia base de treino, componentes = 50:    92.1
Acurácia base de teste, componentes = 50:     82.3
--
Acurácia original na base de treino:    97.6
Acurácia original na base de teste:     88.0
CPU times: total: 7.83 s
Wall time: 2.3 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

> Todas as acurácias são inferiores a da base original (que possui 561 componentes).

> EM relação ao grid search. A acurácia, em ambas as bases, aumentou na medida em que os componentes aumentaram. no Entanto, entre 10 ou 50 componentes não há variação na base de teste. Ambos pontuaram 82.3.

> O tempo de processamento foi reduzido. COntudo, como a estratégia adotada foi de grid search, fica difícil avaliar o tempo para cada possibilidade da lista de componentes.